In [119]:
import sqlite3
from sqlite3 import Error
import os.path
from os import path

from create_db import create_connection, create_table, clean_column_names
from populate_db import extract_data, insert_records
import query_db as qd
import config
import build_dataset as build

import importlib
import datetime
import re
import numpy as np
import pandas as pd


In [120]:
importlib.reload(build)
importlib.reload(config)

<module 'config' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/config.py'>

In [121]:
dataset_1 = pd.read_csv('../data/dataset_main_active1.csv')
dataset_3 = pd.read_csv('../data/dataset_main_active3.csv')

/Users/daminisharma/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21,22,29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [122]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [123]:
dataset_1.head(2)

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,PRIMARY_OFFENSE_CODE,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate,INFRACTION_PER_SENT,felon_count,misd_count,Primary offense code_x,Description (if needed)_x,Recidivate_Risk_Level,Needed a check?_x,Recidivate_Risk_Level_Lenient,Recidivate_Risk_Level_Harsh,Primary offense code_y,Description (if needed)_y,Current_Offense_Risk_Level,Needed a check?_y,Current_Offense_Risk_Level_Lenient,Current_Offense_Risk_Level_Harsh
0,4,AA,1983-07-12,SELL SCHEDULE II,EXPIRED,1984-07-11,1984-07-11,INACTIVE,INACTIVE,1984-07-11,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1961-10-15,ALASKA,UNKNOWN,BORN IN U.S.,SELL SCHEDULE II,NONE,NaN,0,0,NaN,0.0,0.0,2.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,SELL SCHEDULE II,Selling drugs on Schedule II - high potential ...,3.0,YES,2.0,4.0
1,6,AA,1973-01-30,WORTHLESS CHECK,EXPIRED,1973-03-28,0001-01-01,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,WORTHLESS CHECK,AB,1973-04-11,WORTHLESS CHECK,1973,0.0,1.0,0.0,0.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0


In [124]:
train_data, test_data, validate_data, active_sentences = build.split_and_process(dataset_1,config)


249
Size of active sentences dataset:  32711
Dataset size:  778681
Total Number of Unique IDs: 419118
Total Number of IDs in Test Data: 83824
Total Number of IDs in Train Data: 268235
Total Number of IDs in Validate Data: 67059
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.20000095438516122
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.7999990456148388
Does Validate Represent 20% of the Train+Validate Data?: 0.20000059649143737
Does Train Represent 80% of the Train+Validate Data?: 0.7999994035085627


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:656: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age_first_offense'] = (df['first_incarceration_date'] - df['BIRTH_DATE']).astype('<m8[Y]')


Index(['Recidivate_Risk_Level', 'RACE', 'GENDER', 'ETHNICITY',
       'CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'age_cat',
       'first_incarceration_date', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
      dtype='object')


In [125]:
# Sanity Check
print('Train and Test Check:')
build.sanity_check(train_data,test_data)

print('Train and Validate Check:')
build.sanity_check(train_data,validate_data)

print('Train and Active Check:')
build.sanity_check(train_data,active_sentences)



Train and Test Check:
Success: Features match
Success: No NAs remain
Train and Validate Check:
Success: Features match
Success: No NAs remain
Train and Active Check:
Success: Features match


# Analysis